# Surrogate model fitting with Keras

Load required libraries

In [ ]:
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_absolute_error,mean_absolute_percentage_error

import scipy
from scipy.stats import beta

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense, LeakyReLU, Activation, Dropout, concatenate
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import ELU, ReLU
from tensorflow.keras.models import Sequential, Model

import copy
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

Load the samples generated by the InteractiveSurrogate notebook.

Split into an input table and an output table.

In [ ]:
samples = pickle.load(open('samples.p', 'rb'))
samples_out = samples['Electricity:Facility']
samples_in = samples.drop('Electricity:Facility', axis=1)

### Preprocessing

Split data into training and testing with the ratio 80% training, 20% testing.

In [ ]:
train_in, test_in, train_out, test_out = train_test_split(samples_in, samples_out, test_size=0.2)
# train_out.describe()

Set up scaling objects that are fitted to the training data and applied to the testing data.

In [ ]:
def avoid_zero(array):
    array = array + array.max()*0.00001
    if array.ndim == 1:
        array = array.to_numpy().reshape(-1, 1)
    return array


scaler_in = MinMaxScaler()
NN_inputs = scaler_in.fit_transform(X=avoid_zero(train_in))
NN_inputs_test = scaler_in.transform(X=avoid_zero(test_in))

scaler_out = StandardScaler() 
NN_outputs = scaler_out.fit_transform(X=avoid_zero(train_out))
NN_outputs_test = scaler_out.transform(X=avoid_zero(test_out))

Plot the initial and scaled input and output values.

In [ ]:
fig = plt.figure()
plt.plot(train_in.iloc[:, 0], train_out, 'r.')
plt.plot(train_in.iloc[:, 1], train_out, 'b.')

fig = plt.figure()
plt.plot(NN_inputs, NN_outputs, '.')

Construct the model structure using Keras.

We add the input layer, a dense layer with 6 nodes and the output layer.

Other options currently commented would add inter-layer normalisation, ReLU units to nodes and dropout.

In [ ]:
num_inputs = len(NN_inputs[0])
print(f'Number of inputs N: {num_inputs}')
num_outputs = len(NN_outputs[0])
print(f'Number of outputs: {num_outputs}')

model = Sequential(name='model')

model.add(keras.Input(shape=(num_inputs,)))
model.add(Dense(num_inputs*3))
# model.add(BatchNormalization())
# model.add(ReLU())
# model.add(Dropout(rate=0.2))
model.add(Dense(num_outputs))

print(model.summary())

Set the model fitting parameters.

Other possible loss functions include 'mean_absolute_percentage_error' and 'mean_squared_error', see https://keras.io/api/losses/

In [ ]:
loss = 'mean_absolute_error'

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10,
    decay_rate=0.96,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

metrics = ['mae', 'mse', 'mape']

model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

Now we can fit the model to the inputs and outputs and visualise the fitting process.

In [ ]:
%%time
history=model.fit( x=NN_inputs, y=NN_outputs, batch_size = 25,epochs=500, verbose=0)

fig = plt.figure()
plt.plot(history.history['loss'])

Define a function to evaluate a dataframe of input values, with pre and ppost scaling.

In [ ]:
def evaluate_surrogate(inputs):  # inputs in the form of a pandas dataframe, with columns representing parameter inputs. The inputs are in the original scale and order of the parameter definition.
    if inputs.ndim == 1:
        inputs = inputs.to_numpy().reshape(-1, 1)
    evaluation = scaler_in.transform(inputs)  # transform from original scale to [0,1]
    evaluation = model(evaluation, training=False)  # Run inputs through the surrogate
    evaluation = scaler_out.inverse_transform(evaluation)  # transform from [0,1] outputs to original scale
    if isinstance(samples_out, pd.DataFrame):
        evaluation = pd.DataFrame(evaluation, columns=samples_out.columns)  # format into a nice dataframe
    else:
        evaluation = pd.DataFrame(evaluation, columns=[samples_out.name])
    return evaluation

Evaluate the test set using the fitted model.

In [ ]:
model_out = evaluate_surrogate(test_in)
model_out.set_index(test_out.index, inplace=True)  # Match the output to the input via indexing
test_out = test_out.to_frame()

A scatter plot of true EnergyPlus output values against surrogate model values.

The solid black line is the desired 1:1 mapping; the dashed lines show 10% error bounds.

In [ ]:
errors = 0.1
plt.plot(test_out.iloc[:, 0], model_out.iloc[:, 0], '.')
xmin, xmax, ymin, ymax = plt.axis()
plt.plot(np.linspace(xmin, xmax), np.linspace(xmin, xmax), 'black')
plt.xlabel("EnergyPlus")
plt.ylabel("Surrogate")

Calculate the R$^2$ score for the overall model on the test set.

In [ ]:
print('R2 Score on test set: ' + str(r2_score(test_out.values.flatten(), evaluate_surrogate(test_in).values.flatten(), multioutput='raw_values')))

Calculate the R$^2$ score on the training set.

In [ ]:
print('R2 Score on training set: ' + str(r2_score(train_out.values.flatten(), evaluate_surrogate(train_in).values.flatten(), multioutput='raw_values')))

Calculate the maximum test error.

In [ ]:
test_errors = model_out-test_out  # surrogate - EnergyPlus, so overprediciton is positive
test_errors_percent = 100*test_errors/(test_out+0.0001)
test_errors_squared = test_errors**2
test_errors.max()